In [1]:
import numpy as np
import cv2
import itertools
import time
from numba import njit,jit,prange,vectorize,int32,float64,int64
import random
import tensorflow as tf
quesize = 10
k = 10

In [2]:
input_size = 256*3
@njit
def preprocessing(a):
    px_train = a.reshape(input_size)
    px_train = np.asarray(px_train).astype(np.int32)
    return px_train  

In [3]:
class WiSARDrp:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos


In [4]:
input_size = 256*3
no_of_rand_pix_selec = 2**(2)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 100*100

In [5]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10000, 3072, 2)


In [6]:
@njit(parallel = True)
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    ct = 0.02
    b=1
        
    #for i in range(len(images)):
    image = images
    actual_lable = lable
            
    total_sum=[]
            
    for ix in range(dis_number):
                
        t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
        sum_of_ram_output = 0
        dis = d[ix]
                
        for i in range(int(nodes)):
            part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
            ratina_for_one_ram = t_ratina[i]
                    
            n = []                                                                
            for pix in ratina_for_one_ram:
                if image[(pix-1)]>=1:
                    n.append(1)
                else:
                    n.append(0)
                    
            num = 0
            for i in range(no_of_rand_pix_selec):
                num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
            address_of_that_ram = (int)(num)
                
            for key in range(len(part)):
                prt = part[key]
                if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                    sum_of_ram_output += 1
                
        total_sum.append(sum_of_ram_output)        
        
    max_sum = 0
    sec_max = 0
    idx = 0
            
    for i in range(len(total_sum)):
        if max_sum < total_sum[i]:
            max_sum = total_sum[i]
            idx = i
                    
    for j in range(len(total_sum)):
        if sec_max < total_sum[j] and j!=idx:
            sec_max = total_sum[j]
                    
    index_of_dis = idx
    if index_of_dis == actual_lable:
        right += 1
    else:
        wrong += 1
            
    if max_sum == sec_max:
        confidence = 0
    else:
        confidence = 1 - float(sec_max)/float(max_sum)
        if confidence < ct:
            b += 1
        
    return right,wrong                
                
                

@njit
def IsFoundinStabilityQueue(stabilityqueue,i,j,pixel):
    stabilityqueue1 = stabilityqueue
    c = np.zeros((quesize,3),dtype=np.int32) 
    b = np.zeros((4,3),dtype=np.int32)
    for queueIndex in range(quesize):
        queuevalue = np.reshape(stabilityqueue1[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0] == pixel[0][0] and  queuevalue[1][0] == pixel[1][0] and  queuevalue[2][0] == pixel[2][0] :
            b = np.roll(stabilityqueue1[i,j,:queueIndex+1,:],3) 
            c[:queueIndex+1] = b
            c[queueIndex+2:] = stabilityqueue1[i,j,queueIndex+2:,:]
            stabilityqueue1[i,j,:,:] = c
            return stabilityqueue1,True
    #print(stabilityqueue2.shape)
    return stabilityqueue1,False    
    
    
    
@njit
def bewis(frame,w,ID,stabilityqueue,out,d,acc_pos):
    #print(w1)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            #print(i)
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            a = np.zeros((256,3))
            a[0:r,0] = 1  
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            #print(a)
            front = np.reshape(stabilityqueue[i,j,0,:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            pixel_no = i*10+j
            #print(111)
            if front[0][0] == pixel[0][0] and  front[1][0] == pixel[1][0] and  front[2][0] == pixel[2][0] :
                w[i,j] += ID[i,j]
                #print(w1[i,j])
                ID[i,j] += 1
                #print(w1[i,j])
                #continue
            #print(333)    
            #stabilityqueue1,found = IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel)
            #print(222)
            if front[0][0] != pixel[0][0] or  front[1][0] != pixel[1][0] or  front[2][0] != pixel[2][0] :
                stabilityqueue, found = IsFoundinStabilityQueue(stabilityqueue,i,j,pixel)
                if found:
                    ID[i,j] = 1
                #print(1)
                else:
                    stabilityqueue[i,j,:,:] = np.roll(stabilityqueue[i,j,:,:],1)
                #print(stabilityqueue1[i,j,0,:].shape)
                    stabilityqueue[i,j,0,:]= np.reshape(pixel,(3,))
                    w[i,j] = 0
                #print(2)
                    ID[i,j] = 1
            
            #print(w1[i,j])       #here i am getting 0
            if w[i,j] > k:
                #print(w1[i,j])
                #print(pixel_no)
                #print(1)
                pixel_train = preprocessing(a)
                #print(pixel_train.shape)
                        
                images = pixel_train
                lable = pixel_no    
        
        #for i in range(len(images)):
            #image = images[i]
            #num = lable[i]
                image = images
                num = lable
                all_ram_of_selected_discriminator = d[num]
                t_ratina = acc_pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
                for i in range((int)(nodes)):
                    part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                
            #threshold = 0               
                    n = []                                                                
                    for ix in range(len(ratina_for_one_ram)):
                        pix = ratina_for_one_ram[ix]
                        if image[(pix-1)]>=1:
                            n.append(1)
                    #threshold = threshold + 1     
                        else:
                            n.append(0)
                
                    num = 0
                    for i in range(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                    address_of_that_ram = (int)(num)
                    for key in range(ram_address_count):
                        index = part[key]
                #if threshold >= 1:          
                        if index[0] == address_of_that_ram:
                            index[1] += 1
                        else:
                            if index[1] != 0:
                                index[1] -= 1

              
                #train_with_bleeching(d,acc_pos,pixel_train,pixel_no)
                #print(pixel.shape)
                #print(pixel.dtype)
                #print(i,j)
                #print(a)
                #print(a.shape)
            #print(w1[i,j])
            #reconstruct colors from neurons and write it on BG image
            pixel_test = preprocessing(a)
            #print(pixel_no)
            right = 0
            wrong = 1
            #right,wrong = test_with_bleaching(d,acc_pos,pixel_test,pixel_no)
            #print(right)
            if right:
                
                out[i,j] = 0.
                #print(out1)
            if wrong:
                #print(5)
                out[i,j] = 1.
                #print(0)




stabilityqueue = np.zeros((100,100,quesize,3),dtype=np.int32)   #240,256  #480,640 for bird video, 720,1280 for E_IP1231_Day_1
w = np.zeros((100,100),dtype=np.int32)
ID = np.zeros((100,100),dtype=np.int32)
cap = cv2.VideoCapture('bird.avi')
out = np.ones((100,100),dtype=np.float64)
n = 0
    #frame_number = 0
while(True):
    #frame = cv2.imread('Stonehenge.jpg')
    ret, frame = cap.read()
        #Display the resulting frame
        #print(frame.shape)
        #if n <= 3:
            #ret, frame1 = cap.read()
            #cv2.imshow('frame1',frame1)
            #frame = frame
            #print(frame1.shape)
            #n += 1
    width = 100
    height = 100
    dim = (width, height)
 
        # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) 
    cv2.imshow('frame',resized)
        #print(frame.shape)
    bewis(resized,w,ID,stabilityqueue,out,d,acc_pos)
        #print(w)
        #print(out)
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(out,kernel,iterations = 1)
    dilation = cv2.dilate(erosion,kernel,iterations = 1)
    cv2.imshow('out',dilation)
        #frame_number += 1
        #print(frame_number)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # When everything done, release the capture
#cap.release()
cv2.destroyAllWindows() 